In [3]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import random
from pymongo import MongoClient  # Importer le client MongoDB

# Paramètres de scraping
start_id = 740857  # ID de départ
valid_categories = {'politique', 'societe', 'international'}

# Liste d'en-têtes User-Agent pour varier les requêtes
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/115.0",
    "Mozilla/5.0 (Linux; Android 10; SM-A505FN) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36",
]

# Limiter le nombre de requêtes simultanées
SEMAPHORE_LIMIT = 20  # Augmenter le nombre de requêtes simultanées

# Connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/")  # Remplacez par votre URL MongoDB
db = client["ChoufTv_BD"]  # Nom de la base de données
collection = db["articles"]  # Nom de la collection

async def fetch_article(session, article_id, semaphore):
    url = f"https://chouftv.ma/press/{article_id}.html"
    async with semaphore:  # Limiter les requêtes simultanées
        try:
            headers = {
                "User-Agent": random.choice(user_agents)
            }
            async with session.get(url, headers=headers, timeout=5) as response:
                if response.status == 404:
                    return None  # Ignorer les articles inexistants
                elif response.status != 200:
                    print(f"⚠️ Erreur avec l'article {article_id}: Statut {response.status}")
                    return None

                html = await response.text()
                soup = BeautifulSoup(html, 'html.parser')

                # Extraction du titre
                title_tag = soup.find('h1')
                if not title_tag:
                    return None
                title = title_tag.text.strip()

                # Extraction du contenu
                content_div = soup.find('div', class_='content-text')
                if not content_div:
                    return None

                paragraphs = content_div.find_all('p')
                content = ' '.join(p.get_text(strip=True) for p in paragraphs)

                # Vérification que le contenu n'est pas vide
                if not content.strip():
                    return None

                # Extraction de la catégorie
                navbar = soup.find('ul', class_='navbar-head')
                if not navbar:
                    return None

                category_link = navbar.find_all('a')[-1]['href']
                category = category_link.split('/')[-1]

                if category not in valid_categories:
                    return None

                # Extraction de la date
                left_info = soup.find('div', class_='left-info')
                time_tag = left_info.find('time') if left_info else None
                date = time_tag['datetime'].strip() if time_tag and time_tag.has_attr('datetime') else (time_tag.get_text(strip=True) if time_tag else "Date non trouvée")

                # Retourner les données de l'article
                return {
                    "title": title,
                    "category": category,
                    "date": date,
                    "content": content,
                    "url": url
                }
        except Exception as e:
            print(f"⚠️ Erreur avec l'article {article_id}: {e}")
            return None
        finally:
            await asyncio.sleep(random.uniform(0.5, 1.5))  # Réduire le délai aléatoire

async def main():
    semaphore = asyncio.Semaphore(SEMAPHORE_LIMIT)  # Limiter les requêtes simultanées
    async with aiohttp.ClientSession() as session:
        article_id = start_id
        valid_article_count = 0
        tasks = []

        while valid_article_count < 5000:  # Extraire jusqu'à 5000 articles valides
            # Créer une tâche pour chaque article
            task = asyncio.create_task(fetch_article(session, article_id, semaphore))
            tasks.append(task)
            article_id -= 1  # Passer à l'article précédent

            # Limiter le nombre de tâches en attente pour éviter une surcharge mémoire
            if len(tasks) >= SEMAPHORE_LIMIT * 2:
                done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)
                for task in done:
                    article = await task
                    if article:
                        # Insérer l'article dans MongoDB
                        collection.insert_one(article)
                        valid_article_count += 1
                        print(f"\r✅ Articles extraits : {valid_article_count}/5000", end="")
                tasks = list(pending)

        # Traiter les tâches restantes
        if tasks:
            results = await asyncio.gather(*tasks)
            for article in results:
                if article:
                    # Insérer l'article dans MongoDB
                    collection.insert_one(article)
                    valid_article_count += 1
                    print(f"\r✅ Articles extraits : {valid_article_count}/5000", end="")

    print(f"\n✅ Extraction terminée : {valid_article_count} articles enregistrés dans MongoDB")

# Exécution du script
if __name__ == "__main__":
    try:
        asyncio.run(main())  # Pour les scripts Python standard
    except RuntimeError:
        # Pour Jupyter Notebook ou environnements avec une boucle d'événements déjà active
        await main()  # Utilisez await directement

✅ Articles extraits : 672/5000⚠️ Erreur avec l'article 728801: 
⚠️ Erreur avec l'article 728794: 
⚠️ Erreur avec l'article 728798: 
⚠️ Erreur avec l'article 728795: 
⚠️ Erreur avec l'article 728784: 
⚠️ Erreur avec l'article 728789: 
⚠️ Erreur avec l'article 728786: 
⚠️ Erreur avec l'article 728783: 
⚠️ Erreur avec l'article 728792: 
⚠️ Erreur avec l'article 728788: 
⚠️ Erreur avec l'article 728791: 
⚠️ Erreur avec l'article 728785: 
⚠️ Erreur avec l'article 728780: 
⚠️ Erreur avec l'article 728774: 
✅ Articles extraits : 701/5000⚠️ Erreur avec l'article 728175: 
⚠️ Erreur avec l'article 728172: 
✅ Articles extraits : 706/5000⚠️ Erreur avec l'article 727979: 
⚠️ Erreur avec l'article 727980: 
⚠️ Erreur avec l'article 727976: 
⚠️ Erreur avec l'article 727970: 
⚠️ Erreur avec l'article 727977: 
⚠️ Erreur avec l'article 727968: 
⚠️ Erreur avec l'article 727971: 
⚠️ Erreur avec l'article 727919: 
⚠️ Erreur avec l'article 727911: 
⚠️ Erreur avec l'article 727913: 
⚠️ Erreur avec l'article 7

C:\Users\hajar\AppData\Local\Temp\ipykernel_4596\2000277961.py:135: RuntimeWarning: coroutine 'main' was never awaited
  await main()  # Utilisez await directement
